In [4]:
LSP_ANNOTATION_FILE = '/home/ubuntu/datasets/lsp/CO_LSP_train2016.json'
LSP_IMAGES_FOLDER = 'http://vision.caltech.edu/~mronchi/data/LSP/images/'
LSP_IMAGES_SERVER_FOLDER = '/static/images/lsp/'

with open( LSP_ANNOTATION_FILE ) as f:
    _lsp_dataset = json.load(f)

IOError: [Errno 2] No such file or directory: '/home/ubuntu/datasets/lsp/CO_LSP_train2016.json'

# LSP mongodb scripting

In [7]:
import json
import sys
from pymongo import MongoClient
_mongo_client = MongoClient()

_mongo_db = _mongo_client.cocoa_depth_lsp

_mongo_coll_1 = _mongo_db.depth_amt_gui_data
_mongo_coll_2 = _mongo_db.keypoint_labels
_mongo_coll_3 = _mongo_db.depth_hit_id2lsp_subj_id
_mongo_coll_4 = _mongo_db.lsp_subj_id2depth_hit_id

_mongo_coll_5 = _mongo_db.depth_amt_gui_workers
_mongo_coll_6 = _mongo_db.depth_amt_gui_blocked_workers

_mongo_coll_7 = _mongo_db.depth_amt_gui_trials_results
_mongo_coll_8 = _mongo_db.depth_amt_gui_hits_results

In [ ]:
cursor = _mongo_coll_8.find({})
for document in cursor:
    # print document
    if document['_hit_comment'] != "":
        # print document['_hit_id']
        print document['_hit_comment']

In [ ]:
worker_ids = set([i['_worker_id'] for i in _mongo_coll_8.find()])

In [ ]:
# Oisin, AA1R5HDNQYVCC, 20 hits
# Matteo, ADU86DR9LRC0G, 16 hits
# Robert, A27877Z7ZEGJPX, 18 hits
# Matteo 2, A1GM8ZHDSD19ZQ, 4 hits ## he has two worker ids for some reason
hits = [i for i in _mongo_coll_8.find({'_worker_id':'A27877Z7ZEGJPX'})]
hit_ids = sorted([h['_hit_id'] for h in hits])


hits = [i for i in _mongo_coll_8.find({'_hit_comment':'robert new 1'})]
hits = [i for i in _mongo_coll_8.find({'_hit_id':10})]

In [46]:
# This is processAssignments modified to read in from mongodb rather than mturk.
# Run it in a python terminal on the server.

from mturk_api import mturk_depth_api_lsp as m

def processAssignments( save = True, savePath = '/home/ubuntu/amt_guis/cocoa_depth/hits/lsp/', hit_name = '', verbose = False ):
    hits = [i for i in _mongo_coll_8.find()]
    for h in hits:
        if h['_worker_id'] == 'A1GM8ZHDSD19ZQ':
            h['_worker_id'] = 'ADU86DR9LRC0G'
    
    assignments_data = hits
    
    #_mtc = MTurkConnection( host = _host )
    #mtc_assignments = getReviewableAssignments()
    #assignments_data = extractAssignmentData( mtc_assignments, verbose )
    
    # Simple filter for only the assignments
    new_assignments_data = []
    for ass_data in assignments_data:
         _trials_results_dict = json.loads( ass_data['_trials_results'] )
         # print _trials_results_dict[_trials_results_dict.keys()[0]].keys()
         if "_lsp_subj_id" not in _trials_results_dict[_trials_results_dict.keys()[0]].keys():
             continue
            
         new_assignments_data.append(ass_data)
        
    assignments_data = new_assignments_data
    
    num_assignments = len(assignments_data)
    
    # store assignments info here, for persistence
    # list containing all the assignments
    _all_assignments = []
    # list containing the assignments that were flagged by the turkers
    _flagged_assignments = []
    # list with the assignments that were rejected
    _rejected_assignments = []
    # list with the assignments that are not rejected nor flagged
    _good_assignments = []
    # list with the assignments were something inexpected on my side happened
    _error_assignments = []
    
    worker_ids = set()
    
    print "===================================================="
    print "Number of Assignments to analyze: [%d]" %(num_assignments)
    print "===================================================="
    
    count = 0
    for ass_data in assignments_data:
        count += 1
        if verbose:
            print " - Assignment [%d/%d]" %(count,num_assigments)
        
        worker_ids.add( ass_data['_worker_id'] )
        
        cleaned_data     = m.cleanAssignmentData( ass_data )
        _polished_data   = cleaned_data[0]
        _error           = cleaned_data[1]
        _hit_reject_flag = cleaned_data[2]
        _hit_flag        = cleaned_data[3]
        
        _all_assignments.append( _polished_data )
        if _error:
            _error_assignments.append( _polished_data )
        else:
            if _hit_reject_flag:
                _rejected_assignments.append( _polished_data )
            else:
                if _hit_flag:
                    _flagged_assignments.append( _polished_data )
                else:
                    _good_assignments.append( _polished_data )
    
    # print out some stats
    print "Distinct workers:               [%d]" % (len(worker_ids),)
    print "Total number of assignments:    [%d]" % (len(_all_assignments),)
    print "Rejected assignments:           [%d]" % (len(_rejected_assignments),)
    print "Flagged assignments:            [%d]" % (len(_flagged_assignments),)
    print "Good assignments:               [%d]" % (len(_good_assignments),)
    print "Error assignments:              [%d]" % (len(_error_assignments),)
    
    return_dict = {
        "_all_assignments":_all_assignments,
        "_rejected_assignments":_rejected_assignments,
        "_flagged_assignments":_flagged_assignments,
        "_good_assignments":_good_assignments,
        "_error_assignments":_error_assignments}
    
    if save:
        if savePath == '':
            if 'MTURK_STORAGE_PATH' in os.environ:
                savePath = os.environ['MTURK_STORAGE_PATH']
            else:
                savePath == './'
        
        if hit_name == '':
            hit_name = 'cocoa_test_completed_' + str(len(_all_assignments)) + '_DepthHITS'
        
        time_stamp = time.strftime( "%Y-%m-%d_%H-%M-%S" )
        
        filename = os.path.join( savePath, hit_name + '_' + time_stamp + ".pkl")
        
        print "Storing created hit data at %s" % (filename)
        with open(filename, 'wb') as f:
            pickle.dump( return_dict, f )
            
    return return_dict




In [ ]:
import time
import os
import pickle
processAssignments()

In [ ]:
# These are the missing hit ids for matteo and me.
matteo
2, 12, 19, 20

reng
5, 10

In [ ]:
_mongo_coll_1.insert({u'_keypoints_bbox': [0, 0, 256, 256], u'_image_keypoints': [110, 21, 2, 137, 43, 2, 86, 58, 2, 151, 39, 2, 89, 94, 2, 146, 45, 2, 114, 88, 2, 127, 117, 2, 87, 118, 2, 169, 163, 2, 119, 162, 2, 142, 214, 2, 114, 219, 2], u'_comps': [[0, 2], [6, 11], [5, 6], [2, 3], [5, 8]], u'_lsp_img_src': u'/static/images/human/human36m_train_0000279782.jpg', u'_lsp_subj_id': 279782, u'_lsp_img_id': 279782})

# Loading Pickle

In [38]:
import pickle
LSP_RAW_RESULT_PATH = "/Users/Robert/Downloads/LSP_FINALIZED_HITS_2018-04-27_17-33-24.pkl"
data = pickle.load(open(LSP_RAW_RESULT_PATH, 'r'))
data = data['_good_assignments'] + data['_flagged_assignments']

In [40]:
subj_id_to_comps = {}

for d in data:
    for t in d['trials']:
        subj_id_to_comps[t['lsp_subj_id']] = [[int(c) for c in comp.split(',')] for comp in t['depth']['keypoint_comparisons_res'].keys()]



In [42]:
for subj_id, comps in subj_id_to_comps.iteritems():
    if len(comps) < 5:
        print subj_id

In [ ]:
[item for item in _mongo_coll_1.find({'_lsp_img_id':1900})]

In [44]:
subj_id_to_comps[1180]

[[9, 10], [2, 10], [1, 4], [9, 13], [0, 3]]